# A2A

**IMPORTANT**: Fror this you need to run the A2A server first.

```bash
./scripts/example_13_a2a_server_1.sh

./scripts/example_13_a2a_server_2.sh

In [ ]:
import rich

from a2a_utils import get_result_text, task, agent_card
from fasta2a.client import A2AClient
from aixtools.agents import get_agent, run_agent

## Clients

In [ ]:
SERVER_1_URL = "http://127.0.0.1:8000"
SERVER_2_URL = "http://127.0.0.1:8001"

client_1 = A2AClient(base_url=SERVER_1_URL)
client_2 = A2AClient(base_url=SERVER_2_URL)

clients = [client_1, client_2]

In [ ]:
# Get agent cards
cards = {}
clients_by_name = {}
for client in clients:
    card = agent_card(client)
    rich.print(card)
    name = card['name']
    cards[name] = card
    clients_by_name[name] = client

## Router agent

In [ ]:
def card2descrition(card):
    """Convert agent card to a description string."""
    descr = f"{card['name']}: {card['description']}\n"
    skills = card.get('skills', [])
    for skill in skills:
        descr += f"\t - {skill['name']}: {skill['description']}\n"
    return descr

In [ ]:
# Agent descriptions
agent_descriptions = [card2descrition(card) for name, card in cards.items()]
agent_descriptions_str = "\n".join(agent_descriptions)

In [ ]:
system_prompt = f"""
Your task is to route the incoming task to the appropriate agent based on the task description.
You have access to the following agents and their respective skills:

{agent_descriptions_str}

NEVER perform the task yourself. Instead, route it to the appropriate agent based on the task description.
Only invoke one agent at a time.
"""
print(system_prompt)

In [ ]:
async def router(agent_name: str, task_description: str) -> str | None:
    """Route the task to the appropriate agent based on the task description."""
    print(f"Routing to agent '{agent_name}', task '{task_description}'")
    client = clients_by_name.get(agent_name)
    if not client:
        raise ValueError(f"Agent {agent_name} not found")
    ret = await task(client, task_description)
    result = get_result_text(ret)
    print(f"Result from agent '{agent_name}': {result}")
    return result

In [ ]:
agent = get_agent(system_prompt=system_prompt, tools=[router])

## Use the "Agentic system"

In [ ]:
messages = []

In [ ]:
prompt = "What is the sum of 40123456789 and 2123456789?"
messages.append(prompt)
ret = await run_agent(agent=agent, prompt=messages, verbose=True, debug=True)
model_output = ret[0]
messages.append(model_output)
print(model_output)

In [ ]:
prompt = "From that last number, substract 246913578"
messages.append(prompt)
ret = await run_agent(agent=agent, prompt=messages, verbose=True, debug=True)
model_output = ret[0]
messages.append(model_output)
print(model_output)

In [ ]:
prompt = "Calculate the area of a circumference of half that radious"
messages.append(prompt)
ret = await run_agent(agent=agent, prompt=messages, verbose=True, debug=True)
model_output = ret[0]
messages.append(model_output)
print(model_output)